# Some Speed Measurements and things

Start by importing some things and preparing some data to be analysed.
We load the instrument definiton from a pickled file. simply because we can. It could be read from the simtel file as well. But this is how its done during exection on storm as well.

In [3]:
import pickle
import gzip
import os
from ctapipe.io import CameraGeometry

In [4]:
WORKING_DIR='./src/'

def load_instrument():
    import os
    p = os.path.join(WORKING_DIR, 'bundled_files', 'instrument.pickle.gz')
    with gzip.open(p, 'rb') as f:
        return pickle.load(f)

c = load_instrument()

tel_id = 38
pix_x = c.pixel_pos[int(tel_id)][0]
pix_y = c.pixel_pos[int(tel_id)][1]
foc = c.optical_foclen[int(tel_id)]

cam_geom = CameraGeometry.guess(optical_foclen=foc, pix_x=pix_x, pix_y=pix_y)

Now use timeit to measure how long it takes to guess camera geometry.
If this takes too long it ould be chached somehow. the simplest solution could involve a lru_cache directive somewhere

In [5]:
%timeit CameraGeometry.guess(optical_foclen=foc, pix_x=pix_x, pix_y=pix_y)

10 loops, best of 3: 52 ms per loop


Use the lru_cache now. This will be much much faster probably

In [6]:
from functools import lru_cache
@lru_cache(maxsize=128)
def get_cam_geom(tel_id):
    pix_x = c.pixel_pos[int(tel_id)][0]
    pix_y = c.pixel_pos[int(tel_id)][1]
    foc = c.optical_foclen[int(tel_id)]
    cam_geom = CameraGeometry.guess(pix_x, pix_y, foc)
    return cam_geom

In [7]:
%timeit get_cam_geom(38)

The slowest run took 824541.47 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 145 ns per loop


Get an event and perform the tailcuts procedure on the ADC sums within the event.

In [8]:
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.io.hessio import hessio_event_source
file_path = 'bundled_files/gamma_test.simtel.gz'
source = hessio_event_source(os.path.join(WORKING_DIR, file_path), max_events=7)
event = next(source)

sums = event.dl0.tel[tel_id].adc_sums[0]

In [9]:
%timeit tailcuts_clean(cam_geom, sums, 1, picture_thresh=10., boundary_thresh=5.)

10 loops, best of 3: 16.3 ms per loop


Now perform hillas parametrization on the cleaned image

In [10]:
from ctapipe.image.hillas import hillas_parameters

mask = tailcuts_clean(cam_geom, sums, 1, picture_thresh=10., boundary_thresh=5.)
sums[mask == 0] = 0
moments = hillas_parameters(cam_geom.pix_x, cam_geom.pix_y, sums)

In [11]:
%timeit hillas_parameters(cam_geom.pix_x, cam_geom.pix_y, sums)

1000 loops, best of 3: 1.4 ms per loop


Test the runtime of the geometric reconstruction thing.

In [17]:
import astropy
def serialize_hillas_moment(moments):
    r = []
    for obj in moments:
        if isinstance(obj, astropy.units.Quantity):
            r.append({'__value__': obj.value, '__unit__': obj.unit.name})
        else:
            r.append({'__value__': obj})
    return r

d = serialize_hillas_moment(moments)

In [18]:
%timeit serialize_hillas_moment(moments)

10000 loops, best of 3: 25.3 µs per loop


In [19]:
from ctapipe.image.hillas import MomentParameters
def deserialize_hillas_moment(moments):
    r = []
    for dct in moments:
        if '__unit__' in dct:
            r.append(dct['__value__']*astropy.units.Unit(dct['__unit__']))
        else:
            r.append(dct['__value__'])
    return MomentParameters._make(r)
deserialize_hillas_moment(d)

MomentParameters(size=1025085.0, cen_x=<Quantity -0.00021568806335144888 m>, cen_y=<Quantity -0.0001287579064364736 m>, length=<Quantity 0.08242715794356519 m>, width=<Quantity 0.08230498333708622 m>, r=<Quantity 0.00025119701260604656 m>, phi=<Quantity -2.603408847342346 rad>, psi=<Quantity 0.8487183025386534 rad>, miss=<Quantity 7.675767118428263e-05 m>, skewness=0.002885725251076512, kurtosis=2.102454367954421)

In [20]:
%timeit deserialize_hillas_moment(d)

1000 loops, best of 3: 260 µs per loop
